In [ ]:
# HELPER FUNCTION FOR CONTRASTIVE LEARNING VISUAL
def analyze_pairwise_distances(z_struct, z_text):
    # normalize
    z_struct = F.normalize(torch.tensor(z_struct), dim=1).numpy()
    z_text = F.normalize(torch.tensor(z_text), dim=1).numpy()

    pos_distances = np.diag(cosine_distances(z_struct, z_text))
    neg_distances = cosine_distances(z_struct, np.roll(z_text, shift=1, axis=0)).diagonal()

    print(f"Mean Positive Pair Distance: {np.mean(pos_distances):.4f}")
    print(f"Mean Negative Pair Distance: {np.mean(neg_distances):.4f}")

    # Optional plot
    plt.hist(pos_distances, bins=50, alpha=0.7, label='Positive Pairs')
    plt.hist(neg_distances, bins=50, alpha=0.7, label='Negative Pairs')
    plt.legend()
    plt.title("Cosine Distances: Positive vs. Negative Pairs")
    plt.xlabel("Cosine Distance")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [1]:
# #define loss function:
# def fitContrastive(structured_encoder, text_encoder, combined_df, singularity_avoiding=False, y_th=1000,
#                    lr=1e-4,temperature=0.1, batch_size=128, epochs=100, debug=True, device="cpu"):

#     def contrastive_loss(struct_embeddings, text_embeddings, temperature=0.1):
        
#         """Cross-modal InfoNCE: (i-th struct, i-th text) are positives."""
#         struct_norm = torch.nn.functional.normalize(struct_embeddings, dim=1)
#         text_norm = torch.nn.functional.normalize(text_embeddings, dim=1)
#         sim_matrix = torch.matmul(struct_norm, text_norm.T) / temperature

#         print("sim_matrix.shape:", sim_matrix.shape)  # should be [B, B]
#         print("labels.shape:", labels.shape)
#         print("labels.max():", labels.max())
#         print("labels.min():", labels.min())

        
#         labels = torch.arange(sim_matrix.size(0)).to(sim_matrix.device)
#         loss_i2t = torch.nn.functional.cross_entropy(sim_matrix, labels)
#         loss_t2i = torch.nn.functional.cross_entropy(sim_matrix.T, labels)
#         return (loss_i2t + loss_t2i) / 2

#     df = combined_df.copy()
#     df = df[df.group.isin([0, 2])]
#     df["label"] = (df.group == 2).astype(int)

#     if debug:
#         print("[DEBUG] Training dataframe preview:")
#         print(df.head())
#         print(df.columns)

#     structured_cols = [c for c in df.columns if c.startswith("embed_")]
#     text_cols = [c for c in df.columns if c.startswith("text_embed")]

#     X_struct_np = df[structured_cols].values
#     X_text_np = df[text_cols].values
#     y_np = df.label.values.astype(int)

#     X_struct_train, X_struct_val, X_text_train, X_text_val, y_train, y_val = train_test_split(
#         X_struct_np, X_text_np, y_np, test_size=0.2, random_state=42, stratify=y_np
#     )

#     X_struct_train = torch.tensor(X_struct_train, dtype=torch.float32).to(device)
#     X_text_train = torch.tensor(X_text_train, dtype=torch.float32).to(device)
#     y_train = torch.tensor(y_train, dtype=torch.int64).to(device)

#     X_struct_val = torch.tensor(X_struct_val, dtype=torch.float32).to(device)
#     X_text_val = torch.tensor(X_text_val, dtype=torch.float32).to(device)
#     y_val = torch.tensor(y_val, dtype=torch.int64).to(device)

#     optimizer = torch.optim.Adam(
#         list(structured_encoder.parameters()) + list(text_encoder.parameters()), lr=lr
#     )

#     num_samples = len(X_struct_train)
#     all_indices = torch.arange(num_samples)

#     loss_history = []
#     acc_history = []
#     mean_pos_cos_history = []
#     mean_neg_cos_history = []

#     for epoch in range(epochs):
#         perm = all_indices[torch.randperm(num_samples)]

#         for start in range(0, num_samples, batch_size):
#             batch_idx = perm[start: start + batch_size]

#             batch_struct_input = X_struct_train[batch_idx]
#             batch_text_input = X_text_train[batch_idx]
#             batch_labels = y_train[batch_idx]

#             struct_embedding = structured_encoder.forward(
#                 batch_struct_input, singularity_avoiding=singularity_avoiding, y_th=y_th
#             )
#             text_embedding = text_encoder.forward(
#                 batch_text_input, singularity_avoiding=singularity_avoiding, y_th=y_th
#             )

#             train_loss = contrastive_loss(struct_embedding, text_embedding, temperature=0.1)
#             optimizer.zero_grad()
#             train_loss.backward()
#             optimizer.step()

#         with torch.no_grad():
#             struct_embedding = structured_encoder.forward(
#                 X_struct_train, singularity_avoiding=singularity_avoiding, y_th=y_th
#             )
#             text_embedding = text_encoder.forward(
#                 X_text_train, singularity_avoiding=singularity_avoiding, y_th=y_th
#             )
#             cosine_sim = torch.nn.functional.cosine_similarity(struct_embedding, text_embedding)
#             predictions = (cosine_sim > 0.5).long()
#             acc = (predictions == y_train).float().mean().item()

#             pos_mask_eval = y_train == 1
#             neg_mask_eval = y_train == 0
#             mean_pos_cos = cosine_sim[pos_mask_eval].mean().item() if pos_mask_eval.any() else 0.0
#             mean_neg_cos = cosine_sim[neg_mask_eval].mean().item() if neg_mask_eval.any() else 0.0

#         loss_history.append(train_loss.item())
#         acc_history.append(acc)
#         mean_pos_cos_history.append(mean_pos_cos)
#         mean_neg_cos_history.append(mean_neg_cos)

#         if debug:
#             print(f"[Epoch {epoch+1:03}] Loss={train_loss.item():.4f} | "
#                   f"Acc={acc:.3f} | "
#                   f"MeanCos+={mean_pos_cos:.3f} | MeanCos-={mean_neg_cos:.3f}")

#     with torch.no_grad():
#         struct_val_embedding = structured_encoder.forward(X_struct_val, singularity_avoiding=singularity_avoiding, y_th=y_th)
#         text_val_embedding = text_encoder.forward(X_text_val, singularity_avoiding=singularity_avoiding, y_th=y_th)

#         val_loss = contrastive_loss(struct_val_embedding, text_val_embedding, temperature=0.1)
#         cosine_val = torch.nn.functional.cosine_similarity(struct_val_embedding, text_val_embedding)
#         val_acc = ((cosine_val > 0.5).long() == y_val).float().mean().item()

#     print(f"training complete – final train Loss: {loss_history[-1]:.4f}, "
#           f"validation Loss: {val_loss:.4f}, validation Acc: {val_acc:.3f}")

#     # plotting
#     epochs_range = np.arange(1, epochs + 1)
#     plt.figure(figsize=(10, 4))

#     plt.subplot(1, 3, 1)
#     plt.plot(epochs_range, loss_history)
#     plt.title("Train Loss")
#     plt.xlabel("Epoch")

#     plt.subplot(1, 3, 2)
#     plt.plot(epochs_range, acc_history)
#     plt.title("Cosine Top-1 Accuracy")
#     plt.xlabel("Epoch")

#     plt.subplot(1, 3, 3)
#     plt.plot(epochs_range, mean_pos_cos_history, label="Positive Pairs")
#     plt.plot(epochs_range, mean_neg_cos_history, label="Negative Pairs")
#     plt.title("Mean Cosine Similarity")
#     plt.xlabel("Epoch")
#     plt.legend()

#     plt.tight_layout()
#     plt.show()

#     return {
#         "structured_encoder": structured_encoder,
#         "text_encoder": text_encoder,
#         "loss_history": loss_history,
#         "accuracy_history": acc_history,
#         "mean_pos_cos_history": mean_pos_cos_history,
#         "mean_neg_cos_history": mean_neg_cos_history,
#         "final_validation_loss": val_loss.item(),
#         "final_validation_accuracy": val_acc}

In [2]:
#new CL function with print statemetns: 
def fitContrastive(structured_encoder, text_encoder, combined_df, singularity_avoiding=False, y_th=1000,
                   lr=1e-4, temperature=0.1, batch_size=128, epochs=100, debug=True, device="cpu"):

    def contrastive_loss(struct_embeddings, text_embeddings, temperature=0.1):
        B = struct_embeddings.size(0)
        if B < 2:
            return None  # Skip tiny batches

        struct_norm = torch.nn.functional.normalize(struct_embeddings, dim=1)
        text_norm = torch.nn.functional.normalize(text_embeddings, dim=1)
        sim_matrix = torch.matmul(struct_norm, text_norm.T) / temperature

        labels = torch.arange(B).to(sim_matrix.device)

        if debug:
            print("sim_matrix.shape:", sim_matrix.shape)
            print("labels.shape:", labels.shape)
            print("labels.max():", labels.max())
            print("labels.min():", labels.min())

        loss_i2t = torch.nn.functional.cross_entropy(sim_matrix, labels)
        loss_t2i = torch.nn.functional.cross_entropy(sim_matrix.T, labels)
        return (loss_i2t + loss_t2i) / 2

    # ─── Preprocessing 
    df = combined_df.copy()
    df = df[df.group.isin([0, 2])]
    df["label"] = (df.group == 2).astype(int)

    if debug:
        print("[DEBUG] Training dataframe preview:")
        print(df.head())
        print(df.columns)

    structured_cols = [c for c in df.columns if c.startswith("embed_")]
    text_cols = [c for c in df.columns if c.startswith("text_embed")]

    X_struct_np = df[structured_cols].values
    X_text_np = df[text_cols].values
    y_np = df.label.values.astype(int)

    X_struct_train, X_struct_val, X_text_train, X_text_val, y_train, y_val = train_test_split(
        X_struct_np, X_text_np, y_np, test_size=0.2, random_state=42, stratify=y_np
    )

    X_struct_train = torch.tensor(X_struct_train, dtype=torch.float32).to(device)
    X_text_train = torch.tensor(X_text_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.int64).to(device)

    X_struct_val = torch.tensor(X_struct_val, dtype=torch.float32).to(device)
    X_text_val = torch.tensor(X_text_val, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.int64).to(device)

    optimizer = torch.optim.Adam(
        list(structured_encoder.parameters()) + list(text_encoder.parameters()), lr=lr
    )

    num_samples = len(X_struct_train)
    all_indices = torch.arange(num_samples)

    loss_history = []
    acc_history = []
    mean_pos_cos_history = []
    mean_neg_cos_history = []

    for epoch in range(epochs):
        perm = all_indices[torch.randperm(num_samples)]

        for start in range(0, num_samples, batch_size):
            batch_idx = perm[start: start + batch_size]

            batch_struct_input = X_struct_train[batch_idx]
            batch_text_input = X_text_train[batch_idx]
            batch_labels = y_train[batch_idx]

            struct_embedding = structured_encoder.forward(
                batch_struct_input, singularity_avoiding=singularity_avoiding, y_th=y_th
            )
            text_embedding = text_encoder.forward(
                batch_text_input, singularity_avoiding=singularity_avoiding, y_th=y_th
            )

            train_loss = contrastive_loss(struct_embedding, text_embedding, temperature=temperature)
            if train_loss is None:
                continue  # skip backward if batch was too small

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        with torch.no_grad():
            struct_embedding = structured_encoder.forward(
                X_struct_train, singularity_avoiding=singularity_avoiding, y_th=y_th
            )
            text_embedding = text_encoder.forward(
                X_text_train, singularity_avoiding=singularity_avoiding, y_th=y_th
            )
            cosine_sim = torch.nn.functional.cosine_similarity(struct_embedding, text_embedding)
            predictions = (cosine_sim > 0.5).long()
            acc = (predictions == y_train).float().mean().item()

            pos_mask_eval = y_train == 1
            neg_mask_eval = y_train == 0
            mean_pos_cos = cosine_sim[pos_mask_eval].mean().item() if pos_mask_eval.any() else 0.0
            mean_neg_cos = cosine_sim[neg_mask_eval].mean().item() if neg_mask_eval.any() else 0.0

        loss_history.append(train_loss.item() if train_loss else 0.0)
        acc_history.append(acc)
        mean_pos_cos_history.append(mean_pos_cos)
        mean_neg_cos_history.append(mean_neg_cos)

        if debug:
            print(f"[Epoch {epoch+1:03}] Loss={loss_history[-1]:.4f} | "
                  f"Acc={acc:.3f} | "
                  f"MeanCos+={mean_pos_cos:.3f} | MeanCos-={mean_neg_cos:.3f}")

    with torch.no_grad():
        struct_val_embedding = structured_encoder.forward(X_struct_val, singularity_avoiding=singularity_avoiding, y_th=y_th)
        text_val_embedding = text_encoder.forward(X_text_val, singularity_avoiding=singularity_avoiding, y_th=y_th)

        val_loss = contrastive_loss(struct_val_embedding, text_val_embedding, temperature=temperature)
        cosine_val = torch.nn.functional.cosine_similarity(struct_val_embedding, text_val_embedding)
        val_acc = ((cosine_val > 0.5).long() == y_val).float().mean().item()

    print(f"training complete – final train Loss: {loss_history[-1]:.4f}, "
          f"validation Loss: {val_loss.item() if val_loss else 0:.4f}, validation Acc: {val_acc:.3f}")

    # plotting
    epochs_range = np.arange(1, epochs + 1)
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 3, 1)
    plt.plot(epochs_range, loss_history)
    plt.title("Train Loss")
    plt.xlabel("Epoch")

    plt.subplot(1, 3, 2)
    plt.plot(epochs_range, acc_history)
    plt.title("Cosine Top-1 Accuracy")
    plt.xlabel("Epoch")

    plt.subplot(1, 3, 3)
    plt.plot(epochs_range, mean_pos_cos_history, label="Positive Pairs")
    plt.plot(epochs_range, mean_neg_cos_history, label="Negative Pairs")
    plt.title("Mean Cosine Similarity")
    plt.xlabel("Epoch")
    plt.legend()

    plt.tight_layout()
    plt.show()

    return {
        "structured_encoder": structured_encoder,
        "text_encoder": text_encoder,
        "loss_history": loss_history,
        "accuracy_history": acc_history,
        "mean_pos_cos_history": mean_pos_cos_history,
        "mean_neg_cos_history": mean_neg_cos_history,
        "final_validation_loss": val_loss.item() if val_loss else 0.0,
        "final_validation_accuracy": val_acc
    }


In [ ]:
# Create dfs for each embedding type
df_structured = pd.DataFrame(embeddings_X_test, columns=[f"embed_{i}" for i in range(128)])
df_text = pd.DataFrame(compressed_text_test_128, columns=[f"text_embed_{i}" for i in range(128)])
df_labels = pd.DataFrame(y_bert_test, columns=["group"])

# verify shape match
assert df_structured.shape[0] == df_text.shape[0] == df_labels.shape[0]

# make single df
combined_df = pd.concat([df_structured, df_text, df_labels], axis=1)

print("[DEBUG] Final combined_df shape:", combined_df.shape)
print(combined_df.head(2))

In [ ]:
#  define your projection size
proj_dim = 64

#  instantiate two KAN models (CPU only)
def move_model_to_device(model, device):
    for param in model.parameters():
        param.data = param.data.to(device)
        if param.grad is not None:
            param.grad = param.grad.to(device)
    for buffer in model.buffers():
        buffer.data = buffer.data.to(device)

device = torch.device("cpu")

structured_encoder = KAN(width=[128, 64, 64], seed=42)
text_encoder = KAN(width=[128, 64, 64], seed=42)

move_model_to_device(structured_encoder, device)
move_model_to_device(text_encoder, device)


# run the contrastive‐learning + KAN probe
results = fitContrastive(
    structured_encoder=structured_encoder,
    text_encoder=text_encoder,
    combined_df=combined_df,
    device="cpu",
    debug=True,
    lr=1e-4,
    batch_size=128,
    epochs=50,
    temperature=0.1
)


In [ ]:
class ProjectionHead(nn.Module):
    def __init__(self, input_dim, proj_dim=128):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, proj_dim))

    def forward(self, x):
        return self.mlp(x)

In [ ]:
def contrastive_loss(z1, z2, temperature=0.1):
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)

    sim_matrix = torch.matmul(z1, z2.T) / temperature
    batch_size = z1.size(0)
    labels = torch.arange(batch_size).to(z1.device)

    loss_1 = F.cross_entropy(sim_matrix, labels)
    loss_2 = F.cross_entropy(sim_matrix.T, labels)

    return (loss_1 + loss_2) / 2

In [ ]:
def train_contrastive(struct_encoded, text_encoded, y_fused, subject_ids,
                      proj_dim=128, temperature=0.1, lr=1e-3, batch_size=256, epochs=30):

    device = struct_encoded.device
    dataset = torch.utils.data.TensorDataset(struct_encoded, text_encoded)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    proj_head_s = ProjectionHead(struct_encoded.shape[1], proj_dim).to(device)
    proj_head_t = ProjectionHead(text_encoded.shape[1], proj_dim).to(device)
    optimizer = torch.optim.Adam(list(proj_head_s.parameters()) + list(proj_head_t.parameters()), lr=lr)

    for epoch in range(epochs):
        total_loss = 0
        proj_head_s.train()
        proj_head_t.train()
        for batch_s, batch_t in loader:
            optimizer.zero_grad()
            z_s = proj_head_s(batch_s)
            z_t = proj_head_t(batch_t)
            loss = contrastive_loss(z_s, z_t, temperature=temperature)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")


In [ ]:

    # Generate final fused embeddings
    proj_head_s.eval()
    proj_head_t.eval()
    with torch.no_grad():
        fused_proj = (proj_head_s(struct_encoded) + proj_head_t(text_encoded)) / 2
        fused_np = fused_proj.cpu().numpy()

    # Package into dataframe
    fused_df = pd.DataFrame(fused_np, columns=[f"fused_{i}" for i in range(fused_np.shape[1])])
    fused_df["group"] = y_fused
    fused_df["subject_id"] = subject_ids

    return fused_df


In [ ]:
fused_df = train_contrastive(
    struct_encoded=structured_encoder(X_struct_tensor),
    text_encoded=text_encoder(X_text_tensor),
    y_fused=y_fused,
    subject_ids=subject_ids,
    proj_dim=128,
    temperature=0.1,
    lr=1e-3,
    batch_size=256,
    epochs=30
)

# evaluate
results, _, _ = train_models(fused_df, report=True)
X_umap = fused_df.drop(columns=["group", "subject_id"]).to_numpy()
fused_umap, _ = umap_embedding(X_umap)
evaluate_kmeans(fused_umap, Y=y_fused, n_clusters=3)

In [ ]:
best_results = []
config_results = []

proj_dims = [64, 128, 256]
temps = [0.05, 0.1, 0.2]
lrs = [1e-3, 5e-4]
batch_size = 256
epochs = 30

for proj_dim, temperature, lr in product(proj_dims, temps, lrs):
    print(f"\n Testing config: proj_dim={proj_dim}, temp={temperature}, lr={lr}")
    
    fused_df = train_contrastive(
        struct_encoded=structured_encoder(X_struct_tensor),
        text_encoded=text_encoder(X_text_tensor),
        y_fused=y_fused,
        subject_ids=subject_ids,
        proj_dim=proj_dim,
        temperature=temperature,
        lr=lr,
        batch_size=batch_size,
        epochs=epochs)

    # classification
    results, _, _ = train_models(fused_df, report=False)
    accuracy = results["Logistic Regression"]["test_acc"]

    # UMAP + clustering
    X_umap = fused_df.drop(columns=["group", "subject_id"]).to_numpy()
    fused_umap, _ = umap_embedding(X_umap, n_components=2)
    sil_score = silhouette_score(fused_umap, y_fused)

    config_results.append({
        "proj_dim": proj_dim,
        "temperature": temperature,
        "lr": lr,
        "accuracy": accuracy,
        "silhouette": sil_score})

    print(f"→ Accuracy: {accuracy:.4f} | Silhouette: {sil_score:.4f}")

# sort by accuracy or silhouette
config_results.sort(key=lambda x: x["accuracy"], reverse=True)
best = config_results[0]

print("\n Best configuration found:")
print(f"  - proj_dim: {best['proj_dim']}")
print(f"  - temperature: {best['temperature']}")
print(f"  - lr: {best['lr']}")
print(f"  - accuracy: {best['accuracy']:.4f}")
print(f"  - silhouette: {best['silhouette']:.4f}")


In [3]:

# Set random seed
torch.manual_seed(42)
np.random.seed(42)

class MultiNegativeDataset(Dataset):
    """Simple dataset that provides multiple negative samples"""
    
    def __init__(self, struct_features, text_features, labels, num_negatives=5):
        self.struct_features = torch.FloatTensor(struct_features)
        self.text_features = torch.FloatTensor(text_features)
        self.labels = torch.LongTensor(labels)
        self.num_negatives = num_negatives
        
        # Group indices by label for negative sampling
        self.label_to_indices = {}
        for idx, label in enumerate(labels):
            label = int(label)
            if label not in self.label_to_indices:
                self.label_to_indices[label] = []
            self.label_to_indices[label].append(idx)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Get anchor (positive pair)
        anchor_struct = self.struct_features[idx]
        anchor_text = self.text_features[idx]
        anchor_label = int(self.labels[idx])
        
        # Get negative text samples (from different labels)
        neg_candidates = []
        for label, indices in self.label_to_indices.items():
            if label != anchor_label:
                neg_candidates.extend(indices)
        
        # Sample negative indices
        neg_idx = np.random.choice(neg_candidates, self.num_negatives, replace=True)
        neg_texts = self.text_features[neg_idx]
        
        return anchor_struct, anchor_text, neg_texts, anchor_label

class SimpleKANEncoder(nn.Module):
    """Simple KAN-based encoder for multimodal contrastive learning"""
    
    def __init__(self, struct_dim=128, text_dim=128, output_dim=64):
        super().__init__()
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # KAN for structured data
        self.struct_encoder = KAN(
            width=[struct_dim, 128, output_dim],
            grid=5,
            k=3,
            device=device,
            seed=42
        )
        
        # KAN for text data
        self.text_encoder = KAN(
            width=[text_dim, 128, output_dim],
            grid=5,
            k=3,
            device=device,
            seed=42
        )
    
    def encode_structured(self, x):
        return self.struct_encoder(x)
    
    def encode_text(self, x):
        return self.text_encoder(x)

def contrastive_loss_with_negatives(struct_emb, pos_text_emb, neg_text_embs, temperature=0.1):
    """
    Contrastive loss with multiple negatives
    
    Args:
        struct_emb: (batch_size, dim) structured embeddings
        pos_text_emb: (batch_size, dim) positive text embeddings
        neg_text_embs: (batch_size, num_negatives, dim) negative text embeddings
        temperature: temperature parameter
    """
    # Normalize embeddings
    struct_emb = F.normalize(struct_emb, p=2, dim=1)
    pos_text_emb = F.normalize(pos_text_emb, p=2, dim=1)
    neg_text_embs = F.normalize(neg_text_embs, p=2, dim=2)
    
    # Positive similarity (batch_size,)
    pos_sim = torch.sum(struct_emb * pos_text_emb, dim=1) / temperature
    
    # Negative similarities (batch_size, num_negatives)
    neg_sim = torch.bmm(
        struct_emb.unsqueeze(1),  # (batch_size, 1, dim)
        neg_text_embs.transpose(1, 2)  # (batch_size, dim, num_negatives)
    ).squeeze(1) / temperature  # (batch_size, num_negatives)
    
    # Compute loss
    # For each sample: -log(exp(pos) / (exp(pos) + sum(exp(negs))))
    pos_exp = torch.exp(pos_sim)
    neg_exp = torch.exp(neg_sim)
    
    loss = -torch.log(pos_exp / (pos_exp + neg_exp.sum(dim=1)))
    
    # Return mean loss and metrics
    with torch.no_grad():
        mean_pos_sim = pos_sim.mean().item() * temperature
        mean_neg_sim = neg_sim.mean().item() * temperature
    
    return loss.mean(), mean_pos_sim, mean_neg_sim

def train_kan_contrastive(combined_df, num_negatives=5, epochs=10, batch_size=128, lr=1e-4):
    """
    Simple training function for KAN contrastive learning
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"[INFO] Using device: {device}")
    
    # Prepare data
    df = combined_df[combined_df.group.isin([0, 2])].copy()
    df['label'] = (df.group == 2).astype(int)
    
    print(f"[INFO] Data shape: {df.shape}")
    print(f"[INFO] Label distribution: {df.label.value_counts().to_dict()}")
    
    # Extract features
    struct_cols = [c for c in df.columns if c.startswith('embed_')]
    text_cols = [c for c in df.columns if c.startswith('text_embed')]
    
    X_struct = df[struct_cols].values
    X_text = df[text_cols].values
    y = df['label'].values
    
    # Train/val split
    X_struct_train, X_struct_val, X_text_train, X_text_val, y_train, y_val = train_test_split(
        X_struct, X_text, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Create datasets
    train_dataset = MultiNegativeDataset(X_struct_train, X_text_train, y_train, num_negatives)
    val_dataset = MultiNegativeDataset(X_struct_val, X_text_val, y_val, num_negatives)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model
    model = SimpleKANEncoder(struct_dim=128, text_dim=128, output_dim=64).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Training history
    history = {
        'train_loss': [], 'val_loss': [],
        'pos_sim': [], 'neg_sim': [],
        'pos_neg_gap': []
    }
    
    # Training loop
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        train_pos_sim = 0
        train_neg_sim = 0
        
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            anchor_struct, anchor_text, neg_texts, _ = batch
            anchor_struct = anchor_struct.to(device)
            anchor_text = anchor_text.to(device)
            neg_texts = neg_texts.to(device)
            
            # Forward pass
            struct_emb = model.encode_structured(anchor_struct)
            pos_text_emb = model.encode_text(anchor_text)
            
            # Process negative texts
            B, K, D = neg_texts.shape
            neg_texts_flat = neg_texts.view(B * K, D)
            neg_emb_flat = model.encode_text(neg_texts_flat)
            neg_emb = neg_emb_flat.view(B, K, -1)
            
            # Compute loss
            loss, pos_sim, neg_sim = contrastive_loss_with_negatives(
                struct_emb, pos_text_emb, neg_emb, temperature=0.1
            )
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_pos_sim += pos_sim
            train_neg_sim += neg_sim
        
        # Validation
        model.eval()
        val_loss = 0
        val_predictions = []
        val_labels = []
        
        with torch.no_grad():
            # Compute cosine similarity for accuracy
            X_struct_val_t = torch.FloatTensor(X_struct_val).to(device)
            X_text_val_t = torch.FloatTensor(X_text_val).to(device)
            
            struct_val_emb = model.encode_structured(X_struct_val_t)
            text_val_emb = model.encode_text(X_text_val_t)
            
            cosine_sim = F.cosine_similarity(struct_val_emb, text_val_emb)
            val_predictions = (cosine_sim > 0.5).cpu().numpy()
            val_labels = y_val
            
            # Compute validation loss
            for batch in val_loader:
                anchor_struct, anchor_text, neg_texts, _ = batch
                anchor_struct = anchor_struct.to(device)
                anchor_text = anchor_text.to(device)
                neg_texts = neg_texts.to(device)
                
                struct_emb = model.encode_structured(anchor_struct)
                pos_text_emb = model.encode_text(anchor_text)
                
                B, K, D = neg_texts.shape
                neg_texts_flat = neg_texts.view(B * K, D)
                neg_emb_flat = model.encode_text(neg_texts_flat)
                neg_emb = neg_emb_flat.view(B, K, -1)
                
                loss, _, _ = contrastive_loss_with_negatives(
                    struct_emb, pos_text_emb, neg_emb, temperature=0.1
                )
                
                val_loss += loss.item()
        
        # Calculate metrics
        n_train = len(train_loader)
        n_val = len(val_loader)
        
        avg_train_loss = train_loss / n_train
        avg_val_loss = val_loss / n_val
        avg_pos_sim = train_pos_sim / n_train
        avg_neg_sim = train_neg_sim / n_train
        pos_neg_gap = avg_pos_sim - avg_neg_sim
        val_acc = accuracy_score(val_labels, val_predictions)
        
        # Store history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['pos_sim'].append(avg_pos_sim)
        history['neg_sim'].append(avg_neg_sim)
        history['pos_neg_gap'].append(pos_neg_gap)
        
        # Print metrics every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"[Epoch {epoch+1:03d}] Loss: {avg_train_loss:.4f} | "
                  f"Val Acc: {val_acc:.3f} | "
                  f"Pos Sim: {avg_pos_sim:.3f} | Neg Sim: {avg_neg_sim:.3f} | "
                  f"Gap: {pos_neg_gap:.3f}")
    
    return model, history

def plot_training_results(history):
    """Simple plotting function for training results"""
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Loss plot
    ax1.plot(epochs, history['train_loss'], 'b-', label='Train Loss')
    ax1.plot(epochs, history['val_loss'], 'r--', label='Val Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Similarity plot
    ax2.plot(epochs, history['pos_sim'], 'g-', label='Positive Pairs', linewidth=2)
    ax2.plot(epochs, history['neg_sim'], 'r-', label='Negative Pairs', linewidth=2)
    ax2.fill_between(epochs, history['pos_sim'], history['neg_sim'], 
                     alpha=0.3, color='gray', label='Gap')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Average Cosine Similarity')
    ax2.set_title('Positive vs Negative Pair Similarities')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print final metrics
    print("\n=== Final Training Metrics ===")
    print(f"Final Train Loss: {history['train_loss'][-1]:.4f}")
    print(f"Final Val Loss: {history['val_loss'][-1]:.4f}")
    print(f"Final Positive Similarity: {history['pos_sim'][-1]:.4f}")
    print(f"Final Negative Similarity: {history['neg_sim'][-1]:.4f}")
    print(f"Final Pos-Neg Gap: {history['pos_neg_gap'][-1]:.4f}")

# Main execution
if __name__ == "__main__":
    # Your exact data loading code
    # For structured data
    X_test_df = pd.DataFrame(embeddings_X_test).reset_index(drop=True)
    y_df = pd.DataFrame(y_test, columns=['group']).reset_index(drop=True)
    group_df = pd.DataFrame(test_group, columns=['subject_id']).reset_index(drop=True)
    X_test_all_df = pd.concat([X_test_df, y_df, group_df], axis=1).fillna(0)
    
    # For text data
    X_text_test_df = pd.DataFrame(compressed_text_test_128).reset_index(drop=True)
    y_text_df = pd.DataFrame(y_bert_test, columns=['group']).reset_index(drop=True)
    group_text_df = pd.DataFrame(test_keys, columns=['subject_id']).reset_index(drop=True)  
    X_text_test_all_df = pd.concat([X_text_test_df, y_text_df, group_text_df], axis=1).fillna(0)
    
    # Create combined dataframe for contrastive learning
    # Rename columns to avoid conflicts
    df_structured = X_test_df.add_prefix('embed_')
    df_text = X_text_test_df.add_prefix('text_embed_')
    
    # Use the group labels from structured data (should be same as text)
    df_labels = y_df
    
    # Include subject_id if needed for tracking
    df_subject = group_df
    
    # Verify shape match
    assert df_structured.shape[0] == df_text.shape[0] == df_labels.shape[0]
    print(f"[INFO] Structured embeddings shape: {embeddings_X_test.shape}")
    print(f"[INFO] Text embeddings shape: {compressed_text_test_128.shape}")
    
    # Make single df
    combined_df = pd.concat([df_structured, df_text, df_labels, df_subject], axis=1)
    print("[DEBUG] Final combined_df shape:", combined_df.shape)
    print("[DEBUG] Columns:", combined_df.columns.tolist()[:5], "...", combined_df.columns.tolist()[-5:])
    print(combined_df.head(2))
    
    # Train the model
    model, history = train_kan_contrastive(
        combined_df,
        num_negatives=5,  # Use 5 negative samples per positive
        epochs=10,
        batch_size=128,
        lr=1e-4
    )
    
    # Plot results
    plot_training_results(history)

NameError: name 'torch' is not defined

In [ ]:
# Create dfs for each embedding type
df_structured = pd.DataFrame(embeddings_X_test, columns=[f"embed_{i}" for i in range(128)])
df_text = pd.DataFrame(compressed_text_test_128, columns=[f"text_embed_{i}" for i in range(128)])
df_labels = pd.DataFrame(y_bert_test, columns=["group"])

# verify shape match
assert df_structured.shape[0] == df_text.shape[0] == df_labels.shape[0]

# make single df
combined_df = pd.concat([df_structured, df_text, df_labels], axis=1)

print("[DEBUG] Final combined_df shape:", combined_df.shape)
print(combined_df.head(2))